In [2]:
import torch
import numpy as np
import datasets
import pandas as pd
import evaluate
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
from tqdm import tqdm
from transformers import BertForSequenceClassification
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding
import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

/home/julia/miniconda3/envs/ssne/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'ConfusionMatrixDisplay'

In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


In [3]:
train_dataset, val_dataset = datasets.load_dataset(
    "csv",
    data_files="../data/train_data.csv",
    split=[datasets.ReadInstruction("train", to=90, unit="%"), datasets.ReadInstruction("train", from_=-10, unit="%")],
)
train_dataset, val_dataset = train_dataset.rename_column("rating", "label"), val_dataset.rename_column(
    "rating", "label"
)

Generating train split: 16392 examples [00:00, 203816.56 examples/s]


In [4]:
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(train_dataset["label"]), y=train_dataset["label"]
)
class_weights = torch.tensor(np.array(class_weights).astype("float32")).to(device)

In [5]:
for class_idx, class_weight in zip(np.unique(train_dataset["label"]), class_weights):
    print(class_idx, class_weight)

0 tensor(2.9041, device='cuda:0')
1 tensor(2.2662, device='cuda:0')
2 tensor(1.8866, device='cuda:0')
3 tensor(0.6817, device='cuda:0')
4 tensor(0.4510, device='cuda:0')


In [6]:
PRETRAINED_MODEL_NAME = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


def tokenize_function(data):
    return tokenizer(data["review"], padding="max_length", truncation=True)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

train_loader = DataLoader(tokenized_train, shuffle=True, batch_size=16)
val_loader = DataLoader(tokenized_val, batch_size=16)

Map: 100%|██████████| 1639/1639 [00:01<00:00, 848.25 examples/s]


In [7]:
model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels=5)
optimizer = Adam(model.parameters(), lr=5e-5)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
num_epochs = 10
loss_fun = nn.CrossEntropyLoss(class_weights)

for epoch in range(num_epochs):
    losses = []
    for batch in tqdm(train_loader):
        labels = batch["label"].to(device)
        batch = {"attention_mask": batch["attention_mask"].to(device), "input_ids": batch["input_ids"].to(device)}
        outputs = model(**batch)
        loss = loss_fun(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
    print(np.mean(losses))

100%|██████████| 923/923 [17:31<00:00,  1.14s/it]


1.1752253911283397


100%|██████████| 923/923 [17:31<00:00,  1.14s/it]


0.8731204209968303


 10%|█         | 93/923 [01:46<15:46,  1.14s/it]

In [1]:
metric = evaluate.load("accuracy")
model.eval()
for batch in val_loader:
    labels = batch["label"].to(device)
    batch = {
        "attention_mask": batch["attention_mask"].to(device),
        "input_ids": batch["input_ids"].to(device),
    }  # , "token_type_ids":batch['token_type_ids'].to(device)}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

NameError: name 'evaluate' is not defined

In [ ]:
import evaluate

accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.jsonld('recall')
f1_metric = evaluate.load('f1')

model.eval()
for batch in val_loader:
    labels = batch["label"].to(device)
    batch = {
        "attention_mask": batch["attention_mask"].to(device),
        "input_ids": batch["input_ids"].to(device),
    }  # , "token_type_ids":batch['token_type_ids'].to(device)}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    accuracy_metric.add_batch(predictions=predictions, references=labels)
    precision_metric.add_batch(predictions=predictions, references=labels)
    recall_metric.add_batch(predictions=predictions, references=labels)
    f1_metric.add_batch(predictions=predictions, references=labels)


accuracy = accuracy_metric.compute()
f1 = f1_metric.compute()
precision = precision_metric.compute()
recall = recall_metric.compute()

In [ ]:
preds = []
references = []
for batch in val_loader:
    labels = batch["label"].to(device)
    batch = {
        "attention_mask": batch["attention_mask"].to(device),
        "input_ids": batch["input_ids"].to(device),
    }  # , "token_type_ids":batch['token_type_ids'].to(device)}

    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    preds.extend((logits.argmax(-1).tolist()))
    references.extend(labels.tolist())

confusion_matrix = evaluate.confusion_matrix(predictions, references)
ConfusionMatrixDisplay(confusion_matrix_train).plot()
plt.show()

In [ ]:
mca_eval = MulticlassAccuracy(num_classes=5, average=None)
mca_average = MulticlassAccuracy(num_classes=5, average='micro')

In [ ]:
mca_eval(torch.tensor(preds), torch.tensor(references))

In [ ]:
mca_average(torch.tensor(preds), torch.tensor(references))


In [ ]:
torch.save(model.state_dict(), "")